# Deploying trained model which is saved on S3  

In [1]:
! pip install -U sagemaker

You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role

region = boto3.session.Session().region_name
role = get_execution_role()

# Create a low-level SageMaker service client.
sagemaker_client = boto3.client('sagemaker', region_name=region)


In [3]:
import argparse
import os
import warnings

import pandas as pd
import pathlib
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display




# Model path and creating estimator 

In [4]:
from sagemaker.tensorflow.model import TensorFlowModel

model_path = "s3://sagemaker-studio-062044820001-7qbctb3w94p/Training/models/1/tensorflow-training-2022-06-02-23-05-08-776/output/model.tar.gz"
model = TensorFlowModel(model_data=model_path, role=role, framework_version="2.6")

# Model Deployment

In [15]:
instance_type = "ml.t2.medium"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


--------!

In [6]:
features_loc = "pre-processed data/"

test_ds = tf.data.experimental.load(features_loc + "test/")

In [7]:
sample = []
labels = []
for spectrogram, label in test_ds:
      sample = spectrogram
      labels = label
        

sample = tf.expand_dims(sample,0)
result = predictor.predict(np.array(sample))
labels = np.array(labels)
labels

array(0)

In [8]:
test_audio = []
test_labels = []

for audio, label in test_ds:
  test_audio.append(audio.numpy())
  test_labels.append(label.numpy())


result = predictor.predict(np.array(test_audio))


In [9]:
result

{'predictions': [[1.00240016, -0.751947463],
  [1.50245214, 0.957009315],
  [-0.341899037, 0.535389543],
  [-3.8799808, 4.34518766],
  [0.424371302, -0.181929886],
  [-5.38188648, 5.07740498],
  [0.11206419, 0.0452605113],
  [0.536811054, -0.609241247],
  [3.29732704, -4.23928928],
  [2.85538745, -1.70956445],
  [-1.23554015, 1.07647276],
  [-3.28618741, 3.45127535],
  [-23.3575897, 22.2575169],
  [0.0505780801, -0.0083726719],
  [0.381298602, -0.0152498558],
  [-0.952332, 0.225670785],
  [1.61861801, -1.18078506],
  [2.78406239, -2.23948932],
  [1.05328834, -0.388755202],
  [1.79230106, -0.215860695],
  [-0.306556165, 0.501138568],
  [-1.36916041, 1.53293729],
  [1.48108208, -1.4128406],
  [0.706999302, -0.0595934615],
  [0.87271297, -1.05297232],
  [1.83780491, -1.49312472],
  [-3.4733789, 3.72256064],
  [1.06866038, -0.999919593],
  [-9.2781477, 8.54385567],
  [-0.0929806903, 0.334278703],
  [-3.91688776, 4.05431032],
  [1.17078006, -1.3455199],
  [0.00892151, -0.289775342],
  [1.21

In [16]:
# Delete endpoint
sagemaker_client.delete_endpoint(EndpointName=predictor.endpoint_name)

{'ResponseMetadata': {'RequestId': '3b4220e2-b444-460f-a6e6-9363fb811e5c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3b4220e2-b444-460f-a6e6-9363fb811e5c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Fri, 03 Jun 2022 09:31:10 GMT'},
  'RetryAttempts': 0}}